In [1]:
!pip install transformers flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 51.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 63.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 73.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting req

In [2]:
# Data processing
import pandas as pd

# Hugging Face model
from transformers import pipeline

# Import flair pre-trained sentiment model
from flair.models import TextClassifier
classifier = TextClassifier.load('en-sentiment')

# Import flair Sentence to process input text
from flair.data import Sentence

# Import accuracy_score to check performance
from sklearn.metrics import accuracy_score

2023-05-21 16:12:52,071 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmp1r9sbk6v


100%|██████████| 253M/253M [00:07<00:00, 35.1MB/s]

2023-05-21 16:12:59,774 copying /tmp/tmp1r9sbk6v to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2023-05-21 16:13:00,386 removing temp file /tmp/tmp1r9sbk6v


In [10]:
data_ = pd.read_csv('text_with_label.csv')
data_.drop(['Unnamed: 0'],axis=1,inplace=True)
data_.head()

,sentence,prediction
0,twitter prior musk takeover talking directly n...,1
1,article say imply states feature turns crashes...,1
2,og musk duck lives wall,0
3,dare_speak way great powerful musk obviously d...,1
4,cannot_wait finally excuse shower douche,0


In [23]:
data_.rename({'prediction':'label'}, axis=1,inplace=True)
data_.dropna()

,sentence,label
0,twitter prior musk takeover talking directly n...,1
1,article say imply states feature turns crashes...,1
2,og musk duck lives wall,0
3,dare_speak way great powerful musk obviously d...,1
4,cannot_wait finally excuse shower douche,0
...,...,...
61277,like brain_dead lmao anything barely relating ...,1
61278,lying agenda_please correct errors anything sa...,1
61279,hard disagree think parody elon_musk sure tend...,1
61280,yeah think many things lining right smaller_sc...,1


In [24]:
data_['label'].value_counts()

1    33848
0    27434
Name: label, dtype: int64

In [16]:
classifier = pipeline(task="zero-shot-classification", 
                      model="facebook/bart-large-mnli",
                      device=0) 

In [33]:
# Put reviews in a list
data_['sentence'] = data_['sentence'].astype(str)
sequences = data_['sentence'].to_list()

# Define the candidate labels 
candidate_labels = ["positive", "negative"]

# Set the hyppothesis template
hypothesis_template = "The sentiment of this comment is {}."

# Prediction results
hf_prediction = classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

# Save the output as a dataframe
hf_prediction = pd.DataFrame(hf_prediction)

# Take a look at the data
hf_prediction.head()

,sequence,labels,scores
0,twitter prior musk takeover talking directly n...,"[negative, positive]","[0.7223168611526489, 0.2776831388473511]"
1,article say imply states feature turns crashes...,"[negative, positive]","[0.9354385137557983, 0.06456148624420166]"
2,og musk duck lives wall,"[negative, positive]","[0.8357816338539124, 0.16421830654144287]"
3,dare_speak way great powerful musk obviously d...,"[negative, positive]","[0.8426045775413513, 0.1573953926563263]"
4,cannot_wait finally excuse shower douche,"[negative, positive]","[0.9933879375457764, 0.006612041033804417]"


In [37]:
# The column for the predicted topic
hf_prediction['hf_prediction'] = hf_prediction['labels'].apply(lambda x: x[0])

# Map sentiment values
hf_prediction['hf_prediction'] = hf_prediction['hf_prediction'].map({'positive': 1, 'negative': 0})

# The column for the score of predicted topic
hf_prediction['hf_predicted_score'] = hf_prediction['scores'].apply(lambda x: x[0])

# The actual labels
hf_prediction['true_label'] = data_['label']

# Drop the columns that we do not need
hf_prediction = hf_prediction.drop(['labels', 'scores'], axis=1)

# Take a look at the data
hf_prediction.head()

,sequence,hf_prediction,hf_predicted_score,true_label
0,twitter prior musk takeover talking directly n...,0,0.722317,1
1,article say imply states feature turns crashes...,0,0.935439,1
2,og musk duck lives wall,0,0.835782,0
3,dare_speak way great powerful musk obviously d...,0,0.842605,1
4,cannot_wait finally excuse shower douche,0,0.993388,0


In [38]:
# Compare Actual and Predicted
accuracy_score(hf_prediction['hf_prediction'], hf_prediction['true_label'])

0.37495512548546067

Wynik jest niski, prawdopodobnie ze względu na źle dobraną wartość w tfidf jako dzielącą pozytywne od negatywnych (>100)